In [9]:
#import packages
#os to fast track terminal use
import os
os.system("pip install pandas spacy")
#spacy for nlp
import spacy
os.system("spacy install en_core_web_md")
nlp = spacy.load('en_core_web_md')
#pandas for dataframes
import pandas as pd
#numpy for hjelp
import numpy as np

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Usage: python -m spacy [OPTIONS] COMMAND [ARGS]...
Try 'python -m spacy --help' for help.

Error: No such command 'install'.
/tmp/ipykernel_16928/4153407557.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
import numpy as np

In [1]:
#get file
#filepath set for sparately added Ucloud data directory
fp = "/work/cds-lang-data/USEcorpus/USEcorpus/"
#os independent filepath

#filepath for hand-in?


In [17]:
#1 text, 1 file

#load it with encoding 
with open(fp+"a1/0100.a1.txt", encoding="latin-1") as f:
    text = f.read()

#convert it with spacy

text_nlp = nlp(text)

# extract values, filter  NOUN, VERB, ADJ, ADV I have to use "i" to  index items, long words confuse me
extract_this = ["NOUN","VERB","ADJ","ADV"]
#empty list for extracts
extract = []
for i in text_nlp:
    if i.pos_ in extract_this:
        extract.append(i.pos_)
    else:
        pass 
#cunt unique values

In [19]:
extract

['NOUN',
 'NOUN',
 'VERB',
 'VERB',
 'NOUN',
 'VERB',
 'ADJ',
 'NOUN',
 'VERB',
 'VERB',
 'NOUN',
 'NOUN',
 'NOUN',
 'VERB',
 'NOUN',
 'VERB',
 'NOUN',
 'NOUN',
 'ADV',
 'VERB',
 'VERB',
 'ADV',
 'NOUN',
 'NOUN',
 'VERB',
 'ADJ',
 'VERB',
 'NOUN',
 'NOUN',
 'ADV',
 'VERB',
 'VERB',
 'NOUN',
 'NOUN',
 'ADJ',
 'NOUN',
 'NOUN',
 'VERB',
 'NOUN',
 'ADV',
 'ADJ',
 'NOUN',
 'NOUN',
 'NOUN',
 'VERB',
 'ADV',
 'ADJ',
 'VERB',
 'ADV',
 'ADV',
 'ADV',
 'NOUN',
 'VERB',
 'VERB',
 'ADJ',
 'VERB',
 'NOUN',
 'VERB',
 'ADJ',
 'ADJ',
 'NOUN',
 'ADV',
 'ADJ',
 'VERB',
 'NOUN',
 'NOUN',
 'ADJ',
 'VERB',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'NOUN',
 'ADJ',
 'NOUN',
 'ADJ',
 'ADJ',
 'VERB',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'VERB',
 'VERB',
 'ADJ',
 'NOUN',
 'VERB',
 'NOUN',
 'VERB',
 'NOUN',
 'VERB',
 'ADJ',
 'NOUN',
 'NOUN',
 'VERB',
 'ADV',
 'ADJ',
 'VERB',
 'NOUN',
 'NOUN',
 'NOUN',
 'ADV',
 'ADJ',
 'ADV',
 'ADV',
 'ADJ',
 'VERB',
 'NOUN',
 'VERB',
 'ADJ',
 'NOUN',
 'NOUN',
 'ADV',
 'VERB',
 'ADJ',
 'N

In [27]:
np.unique(extract, return_counts= True )

(array(['ADJ', 'ADV', 'NOUN', 'VERB'], dtype='<U4'),
 array([ 57,  38, 109,  87]))

In [ ]:
#sorted(os.listdir(filepath))
